# 1. 获取所有的回复

## 1.1 定义爬虫变量

In [ ]:
url = "https://www.v2ex.com/t/954786"
page_count = 5

## 1.2 开始爬取

In [ ]:
import requests
from bs4 import BeautifulSoup


def has_id_and_class_cell(tag):
    return tag.has_attr('id') and 'cell' in tag.get('class', [])


result = {}
count = 0

for i in range(1, page_count+1):
    response = requests.get(url+"?p={}".format(i))
    soup = BeautifulSoup(response.text, 'html.parser')
    # 回复区域
    reply_area = soup.select(".box")[4]
    # 获取本页所有的回复
    all_reples = reply_area.find_all(has_id_and_class_cell)
    for reply in all_reples:
        username = reply.a.text
        content = reply.select(".reply_content")[0].text
        count += 1
        if not result.get(username):
            result[username] = content
        else:
            print("{}:{}与{}:{}冲突，将使用后者邮件抽奖".format(
                username, result[username], username, content))
            result[username] = content


## 1.3 检查回复数量

In [ ]:
print("回复数量: {}".format(count))
print("用户名去重后数量: {}".format(len(result)))

## 1.4 手动修正

In [ ]:
print("回复数量: {}".format(count))
print("用户名去重后数量: {}".format(len(result)))

# 2. 提取电子邮件

## 2.1 直接提取email

In [ ]:
import re

email_result = {}
email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'

for key, value in result.items():
    matches = re.findall(email_pattern, value)
    if len(matches) > 0:
        email_result[key] = matches[0]

print(email_result)

## 2.2 从Base64获取email

In [ ]:
import base64

base64_pattern = r'[A-Za-z0-9+/=]+'
exception_count = 0
success_count = 0
total_count = 0

for key, value in result.items():
    total_count+=1
    try:
        matches = re.findall(r'[A-Za-z0-9+/=]+', value)
        base64_string = ""
        if len(matches) > 0:
            base64_string = matches[0]
        else:
            base64_string = value
        bytes = base64.b64decode(base64_string)
        string = bytes.decode("utf-8")
        if len(re.findall(email_pattern, string)) > 0:
            email_result[key] = string
            success_count+=1
        else:
            print("{}的回复内容{}的解析值{}不符合email的格式-请手动修正".format(key, value, string))
            exception_count+=1
    except ValueError as e:
        print("{}的回复内容{}解析失败-请手动修正".format(key, value))
        exception_count+=1


## 2.3 手动修正

In [ ]:
def decode(base64_string):
    bytes = base64.b64decode(base64_string)
    return bytes.decode("utf-8")

email_result["Rany"]="rany8864@gmail.com"
email_result["XiaTing19991222"]="1490114698@qq.com"
email_result["xiaoshagua"]="173265155@qq.com"
email_result["craiet"]="cooloyty@gmail.com"
email_result["ihhhkz"]="1427694267@qq.com"
email_result["mtdlr"]="2421654504@qq.com"
email_result["Xyg12133617"]="1272403736@qq.com"



## 2.4 打印最终结果

In [ ]:
print("success_count: {}".format(success_count))
print("exception_count: {}".format(exception_count))
print("success_count+exception_count: {}".format(success_count+exception_count))
print("total_count: {}".format(total_count))
for key, value in email_result.items():
    print("{}:{}".format(key, value))

# 3. 抽奖

In [ ]:
import random

keys = random.sample(list(email_result), 40)
code = []

with open('/Users/tinuv/Developer/Jupyter/兑换码.txt', 'r') as file:
    for line in file:
        code.append(line)

final_result = {key: email_result[key] for key in keys}
count = 0

for key, value in final_result.items():
    line = "{}:{}:{}".format(key, value, code[count])
    count += 1
    with open('中奖名单1.txt', 'a') as f:
        f.write("{}:{}{}".format(key, value,"\n"))
    with open('中奖名单_带兑换码.txt', 'a') as f:
        f.write(line)
    


# 4. 发送邮件

In [ ]:
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

username = os.environ.get('GMAIL_EMAIL')
password = os.environ.get('GMAIL_PASSWORD')

server = smtplib.SMTP('smtp.gmail.com', 587)
server.starttls()
server.login(username, password)

with open('/Users/tinuv/Developer/Jupyter/中奖名单_带兑换码.txt', 'r') as file:
    for line in file:
        try:
            username = line.split(":")[0]
            email = line.split(":")[1]
            code = line.split(":")[2]
            # 创建邮件对象
            msg = MIMEMultipart()
            msg['From'] = username
            msg['To'] = email
            msg['Subject'] = 'DLPlay兑换码(V2EX抽奖奖品)'

            body ="""
            V2EX用户:{},🎉恭喜🎉你在DLPlay抽奖活动(https://www.v2ex.com/t/954786)中幸运中奖!
            DLPlay兑换码:{}
            请在手机的app store中登录与Apple TV相同账号中兑换,然后在Apple TV下载本软件。
            请注意不要在国区兑换(国区兑换需要卡bug)
            """.format(username,code)
            msg.attach(MIMEText(body, 'plain'))
            text = msg.as_string()
            server.sendmail(username, email, text)
        except:
            print("用户{}发送邮件失败，请人工核对!".format(username))
# 关闭连接
server.quit()






